## Dependencies

In [6]:
# Custom data handling methods
import prep_IO

In [7]:
arc_data = prep_IO.arc_data
len(arc_data)

2106968

## Sample info

In [8]:
sample_range = [
    '2020-08-01', '2020-08-02', '2020-08-03', '2020-08-04', '2020-08-05',
    '2020-08-06', '2020-08-07', '2020-08-08', '2020-08-09', '2020-08-10',
    '2020-08-11', '2020-08-12', '2020-08-13', '2020-08-14', '2020-08-15',
    '2020-08-16', '2020-08-17', '2020-08-18', '2020-08-19', '2020-08-20',
    '2020-08-21', '2020-08-22', '2020-08-23', '2020-08-24', '2020-08-25',
    '2020-08-26', '2020-08-27', '2020-08-28', '2020-08-29', '2020-08-30',
    '2020-08-31'
    ]
sample_dates = []
sample_n = 5

for i in range(len( sample_range )):
    date = sample_range[i]
    sample_date = prep_IO.format_acct_file( date )
    sample_dates.append( sample_date )

## Prep hostfile data

In [10]:
# collect all unique (host, s, e, jobid) from date.txt acct file
#search_set_A = []
#
#for sample_date in sample_dates:
#    full_set = prep_IO.acct_file_to_searchset( full_txt=sample_date )
#    
#    for val in full_set:
#        search_set_A.append( val )
#    
#len(search_set_A)

696883

In [11]:
#sample_set_A = prep_IO.get_sample_hosts( search_set_A, sample_n )

### Sample set A

In [15]:
#sample_hosts_A = prep_IO.deep_search_host_broken( sample_set_A, jobids=True )
#sample_hosts_A

In [16]:
# pickle obj
#prep_IO.quick_save( sample_hosts_A, label='sample_A' )

### Sample set B

In [12]:
sample_hosts_B = {('comet-02-35',
  '2020-08-06T13:33:54',
  '2020-08-06T13:51:24',
  '35140328'): {'Source': ['/oasis/projects/nsf/sys200/stats/xsede_stats/archive/comet-02-35.sdsc.edu/1596720622.gz'], 'Single JobID': False},
 ('comet-03-66',
  '2020-08-06T04:02:37',
  '2020-08-06T11:58:20',
  '35132501'): {'Source': ['/oasis/projects/nsf/sys200/stats/xsede_stats/archive/comet-03-66.sdsc.edu/1596678020.gz'], 'Single JobID': False},
 ('comet-06-68',
  '2020-07-06T02:49:32',
  '2020-07-06T02:58:04',
  '34540202'): {'Source': ['/oasis/projects/nsf/sys200/stats/xsede_stats/archive/comet-06-68.sdsc.edu/1593999642.gz'], 'Single JobID': False},
 ('comet-14-07',
  '2020-07-06T20:33:26',
  '2020-07-06T20:33:43',
  '34558370'): {'Source': ['/oasis/projects/nsf/sys200/stats/xsede_stats/archive/comet-14-07.sdsc.edu/1594086085.gz'], 'Single JobID': False},
 ('comet-06-48',
  '2020-06-06T19:55:12',
  '2020-06-06T21:35:34',
  '33954437'): {'Source': ['/oasis/projects/nsf/sys200/stats/xsede_stats/archive/comet-06-48.sdsc.edu/1591494037.gz'], 'Single JobID': False},
 ('comet-01-40',
  '2020-06-06T08:46:31',
  '2020-06-06T08:48:03',
  '33946702'): {'Source': ['/oasis/projects/nsf/sys200/stats/xsede_stats/archive/comet-01-40.sdsc.edu/1591407568.gz'], 'Single JobID': False},
 ('comet-10-16',
  '2020-05-06T15:03:23',
  '2020-05-06T15:12:57',
  '33176106'): {'Source': ['/oasis/projects/nsf/sys200/stats/xsede_stats/archive/comet-10-16.sdsc.edu/1588729257.gz'], 'Single JobID': False},
 ('comet-02-06',
  '2020-05-06T08:21:59',
  '2020-05-06T08:33:23',
  '33162010'): {'Source': ['/oasis/projects/nsf/sys200/stats/xsede_stats/archive/comet-02-06.sdsc.edu/1588729205.gz'], 'Single JobID': False},
 ('comet-07-29',
  '2020-04-05T20:29:23',
  '2020-04-06T06:50:39',
  '32436703'): {'Source': ['/oasis/projects/nsf/sys200/stats/xsede_stats/archive/comet-07-29.sdsc.edu/1586137232.gz'], 'Single JobID': False},
 ('comet-21-61',
  '2020-04-06T20:18:39',
  '2020-04-06T20:21:09',
  '32459913'): {'Source': ['/oasis/projects/nsf/sys200/stats/xsede_stats/archive/comet-21-61.sdsc.edu/1586223732.gz'], 'Single JobID': False},
 ('comet-26-19',
  '2020-03-30T22:17:08',
  '2020-03-30T22:37:00',
  '32342793'): {'Source': ['/oasis/projects/nsf/sys200/stats/xsede_stats/archive/comet-26-19.sdsc.edu/1585618963.gz'], 'Single JobID': False},
 ('comet-06-72',
  '2020-03-30T04:52:58',
  '2020-03-30T06:10:05',
  '32328452'): {'Source': ['/oasis/projects/nsf/sys200/stats/xsede_stats/archive/comet-06-72.sdsc.edu/1585532431.gz'], 'Single JobID': False}}

In [13]:
keys_B = list( sample_hosts_B.keys() )
keys_B

[('comet-02-35', '2020-08-06T13:33:54', '2020-08-06T13:51:24', '35140328'),
 ('comet-03-66', '2020-08-06T04:02:37', '2020-08-06T11:58:20', '35132501'),
 ('comet-06-68', '2020-07-06T02:49:32', '2020-07-06T02:58:04', '34540202'),
 ('comet-14-07', '2020-07-06T20:33:26', '2020-07-06T20:33:43', '34558370'),
 ('comet-06-48', '2020-06-06T19:55:12', '2020-06-06T21:35:34', '33954437'),
 ('comet-01-40', '2020-06-06T08:46:31', '2020-06-06T08:48:03', '33946702'),
 ('comet-10-16', '2020-05-06T15:03:23', '2020-05-06T15:12:57', '33176106'),
 ('comet-02-06', '2020-05-06T08:21:59', '2020-05-06T08:33:23', '33162010'),
 ('comet-07-29', '2020-04-05T20:29:23', '2020-04-06T06:50:39', '32436703'),
 ('comet-21-61', '2020-04-06T20:18:39', '2020-04-06T20:21:09', '32459913'),
 ('comet-26-19', '2020-03-30T22:17:08', '2020-03-30T22:37:00', '32342793'),
 ('comet-06-72', '2020-03-30T04:52:58', '2020-03-30T06:10:05', '32328452')]

In [14]:
# pickle obj
prep_IO.quick_save( sample_hosts_B, label='sample_data_B' )

## A. cpi variance within calendar day
- 1 set equal weights
- 1 set consider length of job

In [24]:
# select date, default='yesterday'
# opt: collect the most recent n days from selected date
#sample_info_A = prep_IO.search_sample_n( sample_set_A, sample_n )
#keys_A = list(sample_info_A['Out'].keys())
#keys_A

In [22]:
# collect each host_file from search set
sample_files_B = { key:[] for key in keys_B }

for key in sample_hosts_B.keys():
    for i in range(len( sample_hosts_B[key]['Source'] )):
        sample_files_B[ key ].append( sample_hosts_B[key]['Source'][i] )

In [23]:
# collect core/inst lines from each file
sample_data_B = prep_IO.find_fixed_set( sample_files_B )

In [24]:
sample_cpiset_B = [ prep_IO.buffer_schema( sample_data_B[ key ][i] )
                   for key in keys_B
                   for i in range(len( sample_data_B[key] )) ]

for i in range(len( sample_cpiset_B )):
    print( f'Len of set {i}:\t', len( sample_cpiset_B[i]['FIXED_CTR0,E,W=48'] ) )

Len of set 0:	 4824
Len of set 1:	 3864
Len of set 2:	 4656
Len of set 3:	 3144
Len of set 4:	 3192
Len of set 5:	 4632
Len of set 6:	 9504
Len of set 7:	 7200
Len of set 8:	 2328
Len of set 9:	 3768
Len of set 10:	 3888
Len of set 11:	 3216


In [27]:
sample_cpiset_B[0].keys()

dict_keys(['FIXED_CTR0,E,W=48', 'FIXED_CTR1,E,W=48', 'FIXED_CTR2,E,W=48\n'])

In [30]:
# pickle obj
prep_IO.quick_save( sample_cpiset_B, label='sample_B' )

#### cpi score
* (core/inst) / 24      -> in cpicore.ipynb, using this base matched result to site for tested sample of jobs
* (core/inst) / len     -> matches avg_cpi() source code
* (core/inst) / 48      -> 'W=48'

#### Average cpi score for key

In [28]:
# compute cpi for each () using core/inst lines
sample_cpi_B1 = fill_cpi( sample_cpiset_B, cpi_hist=False )
sample_cpi_B1

{0: 3.732120006016983,
 1: 49.796406042485714,
 2: 4.321047887667931,
 3: 3.3528618133662005,
 4: 2.7911615120585527,
 5: 7.384950107186035,
 6: 3.357111107424315,
 7: 6.257150413215775,
 8: 3.9184881328188945,
 9: 3.9430920880026075,
 10: 4.021163641270554,
 11: 45.221218744631855}

#### (core/inst) by cycle

In [29]:
sample_cpi_B2 = fill_cpi( sample_cpiset_B, cpi_hist=True )
sample_cpi_B2[0]

{0: 1.8137692151129297,
 1: 0.9392416048315636,
 2: 1.674530177812166,
 3: 1.617187965315779,
 4: 7.638061818204009,
 5: 1.7276395346021725,
 6: 1.7506307221430524,
 7: 40.45253476951104,
 8: 1.361524830876621,
 9: 0.743557967891476,
 10: 0.8857323868635523,
 11: 5.560478813366178,
 12: 1.9793641923105705,
 13: 1.0391791363819283,
 14: 0.9552002500404743,
 15: 1.905649976708307,
 16: 1.5707249663395384,
 17: 3.073729871590453,
 18: 1.691927521539549,
 19: 0.8376766779942549,
 20: 2.2838528427302514,
 21: 1.3121544293071117,
 22: 1.1281354488514956,
 23: 324.71128846142597,
 24: 1.8320926967647866,
 25: 0.9665421991999239,
 26: 1.6969686577231624,
 27: 1.638025312366377,
 28: 5.618617487924883,
 29: 1.5680518212249965,
 30: 1.5727008513950547,
 31: 9.597767396707232,
 32: 1.278245378437157,
 33: 0.7682606241776342,
 34: 0.8594650518655214,
 35: 5.211850911158198,
 36: 1.7739658742048177,
 37: 1.066742042000671,
 38: 0.9417941871638801,
 39: 1.8001019792999373,
 40: 1.4964348139145214,
 

## B. cpi variance between hosts
- 1 set equal weights
- 1 set consider number of jobids tasked

In [ ]:
# select date, default='yesterday'
# opt: collect the most recent n days from selected date
sample_hosts_B = find_recent_hosts( recent, stop, hosts_dict )

In [ ]:
# collect core/inst lines from each host file labeled with range of n days
sample_data_B = find_fixed_set( sample_hosts_B, lim=sample_n )

In [ ]:
# compute cpi for each host using core/inst lines
sample_cpi_B = fill_cpi( sample_data_B )